In [69]:
import math
import numpy as np
import pandas as pd

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

In [70]:
import physics as ph
from Sun import SSM18 as s

In [71]:
pi = math.pi
Z = s.Z

In [72]:
def load_model(epoch):
    filename = 'Mass/1epoch%.02d.csv' % epoch
    return pd.read_csv(filename)
epoch = 0
df1 = load_model(epoch)
dimM = len(df1.index)
df1

,Unnamed: 0,Mass,Radius,Temperature,Density,Luminosity,Hydrogen,Opacity,Convection,Pressure,Velocity
0,0,0.000000e+00,0.000000e+00,1.333000e+07,94.554471,0.000000e+00,0.732,1.318613,0.0,1.730205e+17,8.561869e-11
1,1,1.989737e+30,1.720827e+09,1.323295e+07,93.267398,2.371017e+31,0.732,1.337635,0.0,1.694228e+17,8.186134e-11
2,2,3.979475e+30,2.170742e+09,1.317560e+07,92.506463,4.667612e+31,0.732,1.348390,0.0,1.673123e+17,7.971258e-11
3,3,5.969212e+30,2.487796e+09,1.312752e+07,91.867973,6.909218e+31,0.732,1.357124,0.0,1.655511e+17,7.795029e-11
4,4,7.958950e+30,2.741213e+09,1.308464e+07,91.297969,9.104061e+31,0.732,1.364692,0.0,1.639865e+17,7.640787e-11
...,...,...,...,...,...,...,...,...,...,...,...
996,996,1.981778e+33,5.478443e+10,8.884676e+05,0.065450,2.461129e+33,0.732,6.506261,1.0,7.982444e+12,2.801593e-22
997,997,1.983768e+33,5.565851e+10,7.779882e+05,0.053620,2.461129e+33,0.732,5.330260,1.0,5.726433e+12,5.617357e-23
998,998,1.985758e+33,5.672587e+10,6.475668e+05,0.040707,2.461129e+33,0.732,4.046600,1.0,3.618575e+12,6.097868e-24
999,999,1.987748e+33,5.818398e+10,4.769614e+05,0.025719,2.461129e+33,0.732,2.556699,1.0,1.683936e+12,1.506867e-25


In [73]:
x = df1['Hydrogen'].values
m = df1['Mass'].values
d = df1['Density'].values
t = df1['Temperature'].values

In [74]:
r = np.zeros(dimM)
l = np.zeros(dimM)
p = np.zeros(dimM)
k = np.zeros(dimM)
v = np.zeros(dimM)

In [75]:
# Вычисляем величины r,l,p,k,c,v из x,m,d,t
r[0] = 0.
l[0] = 0.
for i in range(1, dimM):
    r[i] = pow(pow(r[i-1], 3) + 3*(m[i] - m[i-1])/4/pi/d[i-1], 1/3) 
    l[i] = l[i-1] + (m[i] - m[i-1])*ph.Etot(0.5*(d[i]+d[i-1]), 0.5*(t[i]+t[i-1]), x[i], 1 - Z - x[i], Z)
for i in range(dimM):
    X = x[i]
    Y = 1 - Z -X
    p[i] = ph.Pressure(d[i], t[i], X, Y, Z)
    k[i] = ph.opacity(d[i], t[i], X, Y, Z)
    v[i] = 4*ph.m_prot*ph.Etot(d[i], t[i], X, Y, Z)/X/ph.dEpp*ph.year

In [76]:
# Вычисляем новые содержание водорода через 1 млрд лет.
x1 = x*np.exp(-1e9*v)
x1

array([0.67193514, 0.67446459, 0.67591541, ..., 0.732     , 0.732     ,
       0.732     ])

In [77]:
a = np.zeros(dimM) # градиент давления
da = np.zeros(dimM) # несоответствие градиента давления
b = np.zeros(dimM) # градиент температуры
br = np.zeros(dimM) # градиент температуры при радиации
bc = np.zeros(dimM) # градиент температуры при конвекции
db = np.zeros(dimM) # несоответствие градиента температуры 

In [33]:
x1 = x*np.exp(-1e9*v)
d1 = d*1.0
t1 = t*1.01

In [34]:
# Вычисляем величины r,l,p,k,c,v из x,m,d,t
r[0] = 0.
l[0] = 0.
for i in range(1, dimM):
    r[i] = pow(pow(r[i-1], 3) + 3*(m[i] - m[i-1])/4/pi/d1[i-1], 1/3) 
    l[i] = l[i-1] + (m[i] - m[i-1])*ph.Etot(0.5*(d1[i]+d1[i-1]), 0.5*(t1[i]+t1[i-1]), x1[i], 1 - Z - x1[i], Z)
for i in range(dimM):
    X = x1[i]
    Y = 1 - Z -X
    p[i] = ph.Pressure(d1[i], t1[i], X, Y, Z)
    k[i] = ph.opacity(d1[i], t1[i], X, Y, Z)
    v[i] = 4*ph.m_prot*ph.Etot(d1[i], t1[i], X, Y, Z)/X/ph.dEpp*ph.year

In [35]:
sa = 0
sb = 0
for i in range(1, dimM-1):
    X = x1[i]
    Y = 1 - Z - X
    mu = ph.MU(X, Y, Z)
    a[i] = -ph.G*m[i]*d[i]/pow(r[i], 2)
    da[i] = (p[i+1] - p[i])/2/(r[i+1] - r[i]) + (p[i] - p[i-1])/2/(r[i] - r[i-1])
    br[i] = -3/16*k[i]*d[i]*l[i]/ph.sigma/pow(t[i], 3)/4/pi/pow(r[i], 2)
    bc[i] = -0.4*mu/ph.kB*ph.m_prot*ph.G*m[i]/pow(r[i], 2)
    if abs(br[i]) < abs(bc[i]):
        b[i] = br[i]
    else:
        b[i] = bc[i]
    db[i] = (t[i+1] - t[i])/2/(r[i+1] - r[i]) + (t[i] - t[i-1])/2/(r[i] - r[i-1])
    sa += pow((da[i] - a[i])/a[i], 2)
    sb += pow((db[i] - b[i])/b[i], 2)
sa, sb    

(0.7898674207351927, 8.187838516384028)

In [32]:
# (0.12016591197387155, 0.5811328853706212) - original
# (0.17469202734280562, 1.1090422800038706) - orig, d*1.01
# (0.23554882232676688, 1.0401302190684383) - orig, t*1.01
# (1.1001481188877398, 8.422140850300503) ep1 
# (0.7893004443891367, 3.6746544040930758) ep1 d*1.01
# (0.7898674207351927, 8.187838516384028) ep1 t*1.01


In [108]:
N = 10 #
M = 100

In [109]:
e = np.zeros(N)
ed = np.zeros(N)
et = np.zeros(N)
kd = np.zeros(N)
kt = np.zeros(N)

In [110]:
x1 = x*np.exp(-1e9*v)


# Вычисляем величины r,l,p,k,c,v из x,m,d,t
r[0] = 0.
l[0] = 0.
for i in range(1, dimM):
    r[i] = pow(pow(r[i-1], 3) + 3*(m[i] - m[i-1])/4/pi/d1[i-1], 1/3) 
    l[i] = l[i-1] + (m[i] - m[i-1])*ph.Etot(0.5*(d1[i]+d1[i-1]), 0.5*(t1[i]+t1[i-1]), x1[i], 1 - Z - x1[i], Z)
for i in range(dimM):
    X = x1[i]
    Y = 1 - Z -X
    p[i] = ph.Pressure(d1[i], t1[i], X, Y, Z)
    k[i] = ph.opacity(d1[i], t1[i], X, Y, Z)
    v[i] = 4*ph.m_prot*ph.Etot(d1[i], t1[i], X, Y, Z)/X/ph.dEpp*ph.year
    
for n in range(N):
    sa = 0
    sb = 0
    for i in range(1 + M*n, M*n + M -1):
        X = x1[i]
        Y = 1 - Z - X
        mu = ph.MU(X, Y, Z)
        a[i] = -ph.G*m[i]*d1[i]/pow(r[i], 2)
        da[i] = (p[i+1] - p[i])/2/(r[i+1] - r[i]) + (p[i] - p[i-1])/2/(r[i] - r[i-1])
        br[i] = -3/16*k[i]*d1[i]*l[i]/ph.sigma/pow(t1[i], 3)/4/pi/pow(r[i], 2)
        bc[i] = -0.4*mu/ph.kB*ph.m_prot*ph.G*m[i]/pow(r[i], 2)
        if abs(br[i]) < abs(bc[i]):
            b[i] = br[i]
        else:
            b[i] = bc[i]
        db[i] = (t1[i+1] - t1[i])/2/(r[i+1] - r[i]) + (t1[i] - t1[i-1])/2/(r[i] - r[i-1])
        sa += pow((da[i] - a[i])/a[i], 2)
        sb += pow((db[i] - b[i])/b[i], 2)
#     print(sa, sb)
    e[n] = 5*sa + sb    

In [111]:
e

array([5.68906675, 2.1697478 , 1.51036359, 1.42626539, 1.49257524,
       1.7655704 , 1.59584113, 1.6342169 , 2.3415129 , 1.6344724 ])

In [112]:
x1 = x*np.exp(-1e9*v)
d1 = d*1.01
t1 = t*1.0

# Вычисляем величины r,l,p,k,c,v из x,m,d,t
r[0] = 0.
l[0] = 0.
for i in range(1, dimM):
    r[i] = pow(pow(r[i-1], 3) + 3*(m[i] - m[i-1])/4/pi/d1[i-1], 1/3) 
    l[i] = l[i-1] + (m[i] - m[i-1])*ph.Etot(0.5*(d1[i]+d1[i-1]), 0.5*(t1[i]+t1[i-1]), x1[i], 1 - Z - x1[i], Z)
for i in range(dimM):
    X = x1[i]
    Y = 1 - Z -X
    p[i] = ph.Pressure(d1[i], t1[i], X, Y, Z)
    k[i] = ph.opacity(d1[i], t1[i], X, Y, Z)
    v[i] = 4*ph.m_prot*ph.Etot(d1[i], t1[i], X, Y, Z)/X/ph.dEpp*ph.year
    
for n in range(N):
    sa = 0
    sb = 0
    for i in range(1 + M*n, M*n + M -1):
        X = x1[i]
        Y = 1 - Z - X
        mu = ph.MU(X, Y, Z)
        a[i] = -ph.G*m[i]*d1[i]/pow(r[i], 2)
        da[i] = (p[i+1] - p[i])/2/(r[i+1] - r[i]) + (p[i] - p[i-1])/2/(r[i] - r[i-1])
        br[i] = -3/16*k[i]*d1[i]*l[i]/ph.sigma/pow(t1[i], 3)/4/pi/pow(r[i], 2)
        bc[i] = -0.4*mu/ph.kB*ph.m_prot*ph.G*m[i]/pow(r[i], 2)
        if abs(br[i]) < abs(bc[i]):
            b[i] = br[i]
        else:
            b[i] = bc[i]
        db[i] = (t1[i+1] - t1[i])/2/(r[i+1] - r[i]) + (t1[i] - t1[i-1])/2/(r[i] - r[i-1])
        sa += pow((da[i] - a[i])/a[i], 2)
        sb += pow((db[i] - b[i])/b[i], 2)
#     print(sa, sb)
    ed[n] = 5*sa + sb    

In [113]:
x1 = x*np.exp(-1e9*v)
d1 = d*1.0
t1 = t*1.01

# Вычисляем величины r,l,p,k,c,v из x,m,d,t
r[0] = 0.
l[0] = 0.
for i in range(1, dimM):
    r[i] = pow(pow(r[i-1], 3) + 3*(m[i] - m[i-1])/4/pi/d1[i-1], 1/3) 
    l[i] = l[i-1] + (m[i] - m[i-1])*ph.Etot(0.5*(d1[i]+d1[i-1]), 0.5*(t1[i]+t1[i-1]), x1[i], 1 - Z - x1[i], Z)
for i in range(dimM):
    X = x1[i]
    Y = 1 - Z -X
    p[i] = ph.Pressure(d1[i], t1[i], X, Y, Z)
    k[i] = ph.opacity(d1[i], t1[i], X, Y, Z)
    v[i] = 4*ph.m_prot*ph.Etot(d1[i], t1[i], X, Y, Z)/X/ph.dEpp*ph.year
    
for n in range(N):
    sa = 0
    sb = 0
    for i in range(1 + M*n, M*n + M -1):
        X = x1[i]
        Y = 1 - Z - X
        mu = ph.MU(X, Y, Z)
        a[i] = -ph.G*m[i]*d1[i]/pow(r[i], 2)
        da[i] = (p[i+1] - p[i])/2/(r[i+1] - r[i]) + (p[i] - p[i-1])/2/(r[i] - r[i-1])
        br[i] = -3/16*k[i]*d1[i]*l[i]/ph.sigma/pow(t1[i], 3)/4/pi/pow(r[i], 2)
        bc[i] = -0.4*mu/ph.kB*ph.m_prot*ph.G*m[i]/pow(r[i], 2)
        if abs(br[i]) < abs(bc[i]):
            b[i] = br[i]
        else:
            b[i] = bc[i]
        db[i] = (t1[i+1] - t1[i])/2/(r[i+1] - r[i]) + (t1[i] - t1[i-1])/2/(r[i] - r[i-1])
        sa += pow((da[i] - a[i])/a[i], 2)
        sb += pow((db[i] - b[i])/b[i], 2)
#     print(sa, sb)
    et[n] = 5*sa + sb    

In [114]:
e, ed, et

(array([5.68906675, 2.1697478 , 1.51036359, 1.42626539, 1.49257524,
        1.7655704 , 1.59584113, 1.6342169 , 2.3415129 , 1.6344724 ]),
 array([4.59687814, 1.12662226, 0.42185784, 0.23026332, 0.17023725,
        0.14891025, 0.1441002 , 0.14247826, 0.14622392, 0.31583616]),
 array([5.35871383, 2.04452212, 1.4350154 , 1.36224479, 1.43000864,
        1.69805105, 1.53380826, 1.57053348, 2.26607856, 1.57859111]))

In [115]:
ed - e

array([-1.09218861, -1.04312555, -1.08850575, -1.19600207, -1.32233798,
       -1.61666015, -1.45174093, -1.49173864, -2.19528898, -1.31863624])

In [116]:
et - e

array([-0.33035292, -0.12522569, -0.07534819, -0.06402061, -0.0625666 ,
       -0.06751935, -0.06203287, -0.06368342, -0.07543434, -0.05588129])

In [117]:
kd = 1 - 0.001*(ed - e)
kt = 1 - 0.001*(et - e)

In [118]:
kd

array([1.00109219, 1.00104313, 1.00108851, 1.001196  , 1.00132234,
       1.00161666, 1.00145174, 1.00149174, 1.00219529, 1.00131864])

In [119]:
kt

array([1.00033035, 1.00012523, 1.00007535, 1.00006402, 1.00006257,
       1.00006752, 1.00006203, 1.00006368, 1.00007543, 1.00005588])

In [120]:
for n in range(N):
    for j in range(M):
        d1[n*N + j] = d[n*N + j]*kd[n]
        t1[n*N + j] = t[n*N + j]*kt[n]

In [121]:
# Вычисляем величины r,l,p,k,c,v из x,m,d,t
r[0] = 0.
l[0] = 0.
for i in range(1, dimM):
    r[i] = pow(pow(r[i-1], 3) + 3*(m[i] - m[i-1])/4/pi/d1[i-1], 1/3) 
    l[i] = l[i-1] + (m[i] - m[i-1])*ph.Etot(0.5*(d1[i]+d1[i-1]), 0.5*(t1[i]+t1[i-1]), x1[i], 1 - Z - x1[i], Z)
for i in range(dimM):
    X = x1[i]
    Y = 1 - Z -X
    p[i] = ph.Pressure(d1[i], t1[i], X, Y, Z)
    k[i] = ph.opacity(d1[i], t1[i], X, Y, Z)
    v[i] = 4*ph.m_prot*ph.Etot(d1[i], t1[i], X, Y, Z)/X/ph.dEpp*ph.year
    
for n in range(N):
    sa = 0
    sb = 0
    for i in range(1 + M*n, M*n + M -1):
        X = x1[i]
        Y = 1 - Z - X
        mu = ph.MU(X, Y, Z)
        a[i] = -ph.G*m[i]*d1[i]/pow(r[i], 2)
        da[i] = (p[i+1] - p[i])/2/(r[i+1] - r[i]) + (p[i] - p[i-1])/2/(r[i] - r[i-1])
        br[i] = -3/16*k[i]*d1[i]*l[i]/ph.sigma/pow(t1[i], 3)/4/pi/pow(r[i], 2)
        bc[i] = -0.4*mu/ph.kB*ph.m_prot*ph.G*m[i]/pow(r[i], 2)
        if abs(br[i]) < abs(bc[i]):
            b[i] = br[i]
        else:
            b[i] = bc[i]
        db[i] = (t1[i+1] - t1[i])/2/(r[i+1] - r[i]) + (t1[i] - t1[i-1])/2/(r[i] - r[i-1])
        sa += pow((da[i] - a[i])/a[i], 2)
        sb += pow((db[i] - b[i])/b[i], 2)
#     print(sa, sb)
    e[n] = 5*sa + sb    

In [122]:
e

array([ 5.00650015, 67.28203922,  2.58020673,  2.37088939,  2.41420857,
        2.74747555,  2.50659949,  2.56707633,  3.43139096,  2.46807166])